In [45]:
import logging
import threading
import time

In [46]:
def thread_function(name, secondary_id):
    logging.info(f'Thread %s:%s is starting' % (name, secondary_id))
    time.sleep(2)
    logging.info(f'Thread %s:%s is ending' % (name, secondary_id))

In [47]:
format = "%(asctime)s: %(message)s"
logging.basicConfig(format=format, level=logging.INFO, datefmt="%H:%M:%S")

In [58]:
def parent_method():
    logging.info("Main    : before creating thread")
#     x = threading.Thread(target=thread_function, args=('one', 1), daemon=True)
    x = threading.Thread(target=thread_function, args=('one', 1))
    logging.info("Main    : before running thread")
    x.start()
    logging.info("Main    : wait for the thread to finish")
#     x.join()
    logging.info("Main    : all done")
    
# Note that daemon setting won't work in ipython. If I had used
# __name__ == '__main__' and then invoked the script, the use of daemon setting can be demo'ed

In [60]:
parent_method()

18:35:14: Main    : before creating thread
18:35:14: Main    : before running thread
18:35:14: Thread one:1 is starting
18:35:14: Main    : wait for the thread to finish
18:35:16: Thread one:1 is ending
18:35:16: Main    : all done


In [123]:
import logging
import threading
import time

def thread_function(name):
    logging.info("Thread %s: starting", name)
    time.sleep(2)
    logging.info("Thread %s: finishing", name)

format = "%(asctime)s: %(message)s"
logging.basicConfig(format=format, level=logging.INFO,
                    datefmt="%H:%M:%S")

threads = list()
for index in range(3):
    logging.info("Main    : create and start thread %d.", index)
    x = threading.Thread(target=thread_function, args=(index,))
    threads.append(x)
    x.start()

for index, thread in enumerate(threads):
    logging.info("Main    : before joining thread %d.", index)
    thread.join()
    logging.info("Main    : thread %d done", index)

10:00:32: Main    : create and start thread 0.
10:00:32: Thread 0: starting
10:00:32: Main    : create and start thread 1.
10:00:32: Thread 1: starting
10:00:32: Main    : create and start thread 2.
10:00:32: Thread 2: starting
10:00:32: Main    : before joining thread 0.
10:00:34: Thread 0: finishing
10:00:34: Main    : thread 0 done
10:00:34: Main    : before joining thread 1.
10:00:34: Thread 1: finishing
10:00:34: Main    : thread 1 done
10:00:34: Main    : before joining thread 2.
10:00:34: Thread 2: finishing
10:00:34: Main    : thread 2 done


In [124]:
# Using a thread pool executor
import concurrent.futures

# [rest of code]

format = "%(asctime)s: %(message)s"
logging.basicConfig(format=format, level=logging.INFO,
                    datefmt="%H:%M:%S")

with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
    executor.map(thread_function, range(3))

10:00:38: Thread 0: starting
10:00:38: Thread 1: starting
10:00:38: Thread 2: starting
10:00:40: Thread 0: finishing
10:00:40: Thread 2: finishing
10:00:40: Thread 1: finishing


In [151]:
# class FakeDatabase:
#     def __init__(self):
#         self.value = 0

#     def update(self, name):
#         logging.info("Thread %s: starting update", name)
#         local_copy = self.value
#         local_copy += 1
#         time.sleep(0.1)
#         self.value = local_copy
#         logging.info("Thread %s: finishing update", name)

In [153]:
class FakeDatabase:
    def __init__(self):
        self.value = 0
        self._lock = threading.Lock()

    def update(self, name):
        with self._lock:
            logging.info("Thread %s: starting update", name)
            local_copy = self.value
            local_copy += 1
            time.sleep(0.1)
            self.value = local_copy
            logging.info("Thread %s: finishing update", name)

In [154]:
format = "%(asctime)s: %(message)s"
logging.basicConfig(format=format, level=logging.INFO,
                    datefmt="%H:%M:%S")

database = FakeDatabase()
logging.info("Testing update. Starting value is %d.", database.value)
with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
    for index in range(2):
        executor.submit(database.update, index)
logging.info("Testing update. Ending value is %d.", database.value)

10:44:44: Testing update. Starting value is 0.
10:44:44: Thread 0: starting update
10:44:44: Thread 0: finishing update
10:44:44: Thread 1: starting update
10:44:45: Thread 1: finishing update
10:44:45: Testing update. Ending value is 2.


In [156]:
import threading

l = threading.RLock()
print("before first acquire")
l.acquire()
print("before second acquire")
l.acquire()
print("acquired lock twice")
l.release()
l.release()

before first acquire
before second acquire
acquired lock twice


In [183]:
import random

SENTINEL = 'SENTINEL'

def producer(pipeline):
    for iteration in range(10):
        message = random.randint(1,100)
        print("Producer generating message {}".format(message))
        pipeline.set_message(message, "Producer")
        print("Producer sent message {}".format(message))
        
    pipeline.set_message(SENTINEL, "Producer")

In [184]:
def consumer(pipeline):
    message = None
    while message is not SENTINEL:
        message = pipeline.get_message("Consumer")        
        print("Consumer received message {}".format(message))

In [185]:
class Pipeline:
    def __init__(self):
        self.message = None
        self.producer_lock = threading.Lock()
        self.consumer_lock = threading.Lock()
        self.consumer_lock.acquire()
        
    def set_message(self, message, agent):
        self.producer_lock.acquire()
        self.message = message
        self.consumer_lock.release()
        
    def get_message(self, agent):
        self.consumer_lock.acquire()
        message = self.message
        self.producer_lock.release()
        return message

In [186]:
if __name__ == "__main__":
    pipeline = Pipeline()
    with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
        executor.submit(producer, pipeline)
        executor.submit(consumer, pipeline)

Producer generating message 95
Producer sent message 95
Producer generating message 64
Consumer received message 95
Producer sent message 64Consumer received message 64
Producer generating message 64
Producer sent message 64
Producer generating message 73

Consumer received message 64
Producer sent message 73
Producer generating message 29
Consumer received message 73
Producer sent message 29Consumer received message 29

Producer generating message 2
Producer sent message 2
Consumer received message 2Producer generating message 84
Producer sent message 84
Producer generating message 31

Consumer received message 84
Producer sent message 31Consumer received message 31

Producer generating message 7
Producer sent message 7
Producer generating message 67
Consumer received message 7Producer sent message 67

Consumer received message 67
Consumer received message SENTINEL


In [6]:
import concurrent.futures
import logging
import queue
import random
import threading
import time

def producer(queue, event):
    """Pretend we're getting a number from the network."""
    while not event.is_set():
        message = random.randint(1, 101)
        logging.info("Producer got message: %s", message)
        queue.put(message)

    logging.info("Producer received event. Exiting")

def consumer(queue, event):
    """Pretend we're saving a number in the database."""
    while not event.is_set() or not queue.empty():
        message = queue.get()
        logging.info(
            "Consumer storing message: %s (size=%d)", message, queue.qsize()
        )

    logging.info("Consumer received event. Exiting")

if __name__ == "__main__":
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO,
                        datefmt="%H:%M:%S")

    pipeline = queue.Queue(maxsize=10)
    event = threading.Event()
    with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
        executor.submit(producer, pipeline, event)
        executor.submit(consumer, pipeline, event)

        time.sleep(0.1)
        logging.info("Main: about to set event")
        event.set()

18:29:14: Producer got message: 28
18:29:14: Producer got message: 45
18:29:14: Consumer storing message: 28 (size=0)
18:29:14: Producer got message: 2
18:29:14: Consumer storing message: 45 (size=0)
18:29:14: Producer got message: 69
18:29:14: Consumer storing message: 2 (size=0)
18:29:14: Producer got message: 33
18:29:14: Consumer storing message: 69 (size=0)
18:29:14: Producer got message: 73
18:29:14: Consumer storing message: 33 (size=0)
18:29:14: Producer got message: 84
18:29:14: Consumer storing message: 73 (size=0)
18:29:14: Producer got message: 30
18:29:14: Consumer storing message: 84 (size=0)
18:29:14: Producer got message: 31
18:29:14: Consumer storing message: 30 (size=0)
18:29:14: Consumer storing message: 31 (size=0)
18:29:14: Producer got message: 60
18:29:14: Producer got message: 58
18:29:14: Producer got message: 88
18:29:14: Producer got message: 53
18:29:14: Consumer storing message: 60 (size=0)
18:29:14: Consumer storing message: 58 (size=2)
18:29:14: Producer 

In [7]:
import threading 
def second_alarm(message):
    print(message)
    
alarm = threading.Timer(1.0, second_alarm, ["Hello world!"])
alarm.start()
alarm.join()

Hello world!


In [19]:
# importing the modules
from threading import *
import time

# creating thread instance where count = 3
obj = Semaphore(3)

# creating instance
def display(name):
    for i in range(5):
        # calling acquire method
        obj.acquire()        
        print('Hello #%d, ' % i + name)
        # calling release method
        obj.release()        
        time.sleep(1)


# creating multiple thread
t1 = Thread(target = display , args = ('Thread-1',))
t2 = Thread(target = display , args = ('Thread-2',))
t3 = Thread(target = display , args = ('Thread-3',))
t4 = Thread(target = display , args = ('Thread-4',))
t5 = Thread(target = display , args = ('Thread-5',))

# calling the threads
t1.start()
t2.start()
t3.start()
t4.start()
t5.start()


Hello #0, Thread-1
Hello #0, Thread-2
Hello #0, Thread-3
Hello #0, Thread-4
Hello #0, Thread-5
Hello #1, Thread-2Hello #1, Thread-1Hello #1, Thread-4
Hello #1, Thread-5


Hello #1, Thread-3
Hello #2, Thread-4Hello #2, Thread-5Hello #2, Thread-2


Hello #2, Thread-1Hello #2, Thread-3

Hello #3, Thread-4Hello #3, Thread-2Hello #3, Thread-5


Hello #3, Thread-1Hello #3, Thread-3

Hello #4, Thread-5
Hello #4, Thread-4Hello #4, Thread-2

Hello #4, Thread-1
Hello #4, Thread-3


In [3]:
# program to demonstrate
# barriers in python
  
import threading
  
barrier = threading.Barrier(3)
  
class thread(threading.Thread):
    def __init__(self, thread_ID):
        threading.Thread.__init__(self)
        self.thread_ID = thread_ID
    def run(self):
        print(str(self.thread_ID) + "\n")
        print("Parties = " + str(barrier.parties) + "\n")
        print("n_waiting = " + str(barrier.n_waiting) + "\n")
        barrier.wait()
          
thread1 = thread(100)
thread2 = thread(101)
  
thread1.start()
thread2.start()

barrier.wait()
        
print(str(barrier.broken) + "\n")
barrier.reset()
print("n_waiting after reset = " + str(barrier.n_waiting))
barrier.abort()
print("End")

100

Parties = 3

n_waiting = 0

101

Parties = 3

n_waiting = 1

False

n_waiting after reset = 0
End
